In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
import pandas as pd

In [ ]:
df1_1= pd.read_csv('/content/gdrive/MyDrive/dataStrom/Historical-transaction-data.csv')
df2=pd.read_csv('/content/gdrive/MyDrive/dataStrom/Store-info.csv')
# print(df1.shape)
df1_1.head(5)

,item_description,transaction_date,invoice_id,customer_id,shop_id,item_price,quantity_sold
0,ORANGE BARLEY 1.5L,2021-12-11T00:00:00.000Z,147.0,BGXA,SHOP008,220,2
1,GINGER BEER 1.5L,2021-10-17T00:00:00.000Z,371.0,IA25,SHOP112,220,2
2,TONIC PET 500ML,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,160,2
3,CREAM SODA 1L,2021-12-13T00:00:00.000Z,484.0,VN7V,SHOP008,150,2
4,STRAWBERRY MILK 180ML,2021-10-23T00:00:00.000Z,1310.0,7S00,SHOP112,210,5


In [ ]:
trans_df=df1_1
# # trans_df=trans_df1.drop('item_description',axis=1)

store_df=df2
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder

# Merge the data on shop_id
df = pd.merge(trans_df, store_df, on='shop_id')

# Convert transaction_date to datetime and extract month and year
df['transaction_date'] = pd.to_datetime(df['transaction_date'])
df['month'] = df['transaction_date'].dt.month
df['year'] = df['transaction_date'].dt.year

In [ ]:
# Encode 'item_description' using One-Hot Encoding
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(df[['item_description']]).toarray())
merged_data = df.join(enc_df)

In [ ]:
merged_data.shape

(473974, 49)

In [ ]:
df=merged_data.drop('item_description',axis=1)
# Drop unnecessary columns
df = df.drop(['invoice_id', 'customer_id', 'transaction_date'], axis=1)

# Scale numerical variables
scaler = StandardScaler()
df[['item_price', 'quantity_sold', 'shop_area_sq_ft']] = scaler.fit_transform(df[['item_price', 'quantity_sold', 'shop_area_sq_ft']])

In [ ]:
df.info()

In [ ]:
dfTest = df[df.isnull().any(axis=1)]
dfTest.head(3)

In [ ]:
dfTest.shape

(86633, 45)

In [ ]:
df = df[df['shop_profile'].notnull()]
df.shape

(387341, 45)

In [ ]:
# Encode categorical variables
le = LabelEncoder()
df['shop_profile'] = le.fit_transform(df['shop_profile'])

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df.drop(['shop_id', 'shop_profile'], axis=1).values, df['shop_profile'].values, test_size=0.2)

In [ ]:
X_train.shape

(309872, 43)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Train a decision tree classifier
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)
dt_y_pred = dt_clf.predict(X_test)
dt_f1 = f1_score(y_test, dt_y_pred, average='macro')
print('Decision Tree F1 score:', dt_f1)

# Train a random forest classifier
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)
rf_y_pred = rf_clf.predict(X_test)
rf_f1 = f1_score(y_test, rf_y_pred, average='macro')
print('Random Forest F1 score:', rf_f1)

# Train a logistic regression classifier
lr_clf = LogisticRegression(random_state=42)
lr_clf.fit(X_train, y_train)
lr_y_pred = lr_clf.predict(X_test)
lr_f1 = f1_score(y_test, lr_y_pred, average='macro')
print('Logistic Regression F1 score:', lr_f1)


Decision Tree F1 score: 0.734208834480794
Random Forest F1 score: 0.7235154268729872
Logistic Regression F1 score: 0.3927129741592426


In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score


# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(25, input_shape=(X_train.shape[1],), activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

# Predict on the test set
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

# Evaluate the model
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)


Epoch 1/15
9684/9684 [==============================] - 33s 3ms/step - loss: 1.3766 - accuracy: 0.4067 - val_loss: 1.2189 - val_accuracy: 0.4066
Epoch 2/15
9684/9684 [==============================] - 23s 2ms/step - loss: 1.0504 - accuracy: 0.4238 - val_loss: 1.0258 - val_accuracy: 0.4634
Epoch 3/15
9684/9684 [==============================] - 25s 3ms/step - loss: 1.0085 - accuracy: 0.4402 - val_loss: 0.9558 - val_accuracy: 0.4712
Epoch 4/15
9684/9684 [==============================] - 25s 3ms/step - loss: 0.9734 - accuracy: 0.4622 - val_loss: 0.9591 - val_accuracy: 0.4800
Epoch 5/15
9684/9684 [==============================] - 25s 3ms/step - loss: 0.9615 - accuracy: 0.4724 - val_loss: 0.9481 - val_accuracy: 0.4756
Epoch 6/15
9684/9684 [==============================] - 26s 3ms/step - loss: 0.9560 - accuracy: 0.4770 - val_loss: 0.9623 - val_accuracy: 0.4903
Epoch 7/15
9684/9684 [==============================] - 24s 2ms/step - loss: 0.9524 - accuracy: 0.4803 - val_loss: 0.9338 - val_ac

In [ ]:

# Train a decision tree classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)
# Make predictions on the test set
y_pred = clf.predict(X_test)
# Print the classification report
print(classification_report(y_test, y_pred))

# assuming y_true and y_pred are the true and predicted values of shop_profile
f1 = f1_score(y_test, y_pred, average='macro')
print('F1 score:', f1)

              precision    recall  f1-score   support

           0       0.83      0.69      0.75     29210
           1       0.66      0.81      0.72     26089
           2       0.72      0.70      0.71     22170

    accuracy                           0.73     77469
   macro avg       0.74      0.73      0.73     77469
weighted avg       0.74      0.73      0.73     77469

F1 score: 0.7292516884167034


## detecting

In [ ]:
dfTest.head(2)

,shop_id,item_price,quantity_sold,shop_area_sq_ft,shop_profile,month,year,0,1,2,...,28,29,30,31,32,33,34,35,36,37
13404,SHOP087,0.079815,0.045575,-0.802843,NaN,10,2021,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13405,SHOP087,-0.279974,0.045575,-0.802843,NaN,10,2021,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
dfTest2=dfTest.drop(['shop_id', 'shop_profile'],axis=1)

In [ ]:
import numpy as np
dt_y_pred = dt_clf.predict(dfTest2.values)
unique_vals, counts = np.unique(dt_y_pred, return_counts=True)

print(counts)
print(unique_vals)

[ 9376 35267 41990]
[0 1 2]


In [ ]:
result = pd.concat([dfTest[['shop_id']], pd.DataFrame({'val':dt_y_pred})], axis=1)
result = result.reset_index(drop=True)
result['profile']=le.inverse_transform(result['val'])
result.head(10)

,shop_id,val,profile
0,SHOP087,2,Moderate
1,SHOP087,2,Moderate
2,SHOP087,2,Moderate
3,SHOP087,2,Moderate
4,SHOP087,2,Moderate
5,SHOP087,2,Moderate
6,SHOP087,2,Moderate
7,SHOP087,2,Moderate
8,SHOP087,2,Moderate
9,SHOP087,2,Moderate


In [ ]:
result['profile'].value_counts()

Moderate    41990
Low         35267
High         9376
Name: profile, dtype: int64

In [ ]:
grouped = result.groupby('shop_id')['profile'].value_counts()
result1 = grouped.unstack(fill_value=0)
result1.columns = ['Low', 'Moderate', 'High']
result1 = result1.reset_index()
result1.head(10)

,shop_id,Low,Moderate,High
0,SHOP002,0,0,4800
1,SHOP019,0,0,3766
2,SHOP023,971,1231,1901
3,SHOP024,55,2822,6
4,SHOP029,0,0,3799
5,SHOP030,0,0,3616
6,SHOP038,0,0,3136
7,SHOP044,0,2641,0
8,SHOP046,0,3438,0
9,SHOP050,0,0,3929


In [ ]:
max_col = result1[['Low', 'Moderate', 'High']].idxmax(axis=1)
result1['state'] = max_col
result1.head(10)

,shop_id,Low,Moderate,High,state
0,SHOP002,0,0,4800,High
1,SHOP019,0,0,3766,High
2,SHOP023,971,1231,1901,High
3,SHOP024,55,2822,6,Moderate
4,SHOP029,0,0,3799,High
5,SHOP030,0,0,3616,High
6,SHOP038,0,0,3136,High
7,SHOP044,0,2641,0,Moderate
8,SHOP046,0,3438,0,Moderate
9,SHOP050,0,0,3929,High


In [ ]:
testFile=pd.read_csv('Testing-data.csv')
testFile.head(3)

,shop_id,shop_profile
0,SHOP046,NaN
1,SHOP024,NaN
2,SHOP023,NaN


In [ ]:
def get_state(shop_id):
    row = result1.loc[result1['shop_id'] == shop_id]
    if len(row) == 0:
        return None
    else:
        return row.iloc[0]['state']

testFile['shop_profile'] = testFile['shop_id'].apply(get_state)

In [ ]:
testFile.head(30)

,shop_id,shop_profile
0,SHOP046,Moderate
1,SHOP024,Moderate
2,SHOP023,High
3,SHOP097,High
4,SHOP044,Moderate
5,SHOP030,High
6,SHOP038,High
7,SHOP029,High
8,SHOP096,Moderate
9,SHOP092,High


In [ ]:
testFile.to_csv('Testing-data.csv',index=False)